# X-ray Inception-V3

## Prepare dataset 

In [1]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [2]:
import pandas as pd
import numpy as np
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.preprocessing.image import ImageDataGenerator
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(r'/home/gxt/study/CT/dataset/train', 
                                                   target_size = (512, 512),
                                                   color_mode = 'rgb',
                                                   batch_size = 16,
                                                   class_mode = 'categorical',
                                                   shuffle = True)
val_generator = val_datagen.flow_from_directory(r'/home/gxt/study/CT/dataset/val', 
                                                   target_size = (512, 512),
                                                   color_mode = 'rgb',
                                                   batch_size = 16,
                                                   class_mode = 'categorical',
                                                   shuffle = True)

test_generator = test_datagen.flow_from_directory(r'/home/gxt/study/CT/dataset/test', 
                                                   target_size = (512, 512),
                                                   color_mode = 'rgb',
                                                   batch_size = 16,
                                                   class_mode = 'categorical',
                                                   shuffle = False)

## Prepare Models 

In [4]:
#from tensorflow.keras.applications.inception_v3 import InceptionV3
# import the necessary packages
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, models, layers
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [5]:
#Add on a couple of custom CNN layers on top of the Inception V3 model. 
baseModel = InceptionV3(weights="imagenet", include_top=False, input_tensor=Input(shape=(512, 512, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# Compose the final model
model = Model(inputs=baseModel.input, outputs=headModel)
#for layer in baseModel.layers:
    #layer.trainable = False

In [ ]:
model.summary()

In [7]:
INIT_LR = 1e-5    # This value is specific to what model is chosen: Inception, VGG or ResNet etc.
EPOCHS = 50
#BS = 8

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', mode='auto', patience=6, verbose=2)

print("Compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
# train the full model, since we unfroze the pre-trained weights above
print("Training the full stack model...")
H = model.fit_generator(generator = train_generator, steps_per_epoch=100,validation_data = val_generator, epochs = EPOCHS)

## Performance 

In [ ]:
Testresults = model.evaluate(test_generator)
print("test loss, test acc:", Testresults)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
target_names=[]
#print(test_generator.classes);
for key in test_generator.class_indices:
    target_names.append(key)
#print(target_names)    
Y_pred = model.predict(test_generator)
#print(Y_pred)
y_pred = np.argmax(Y_pred,axis=1)
#print(y_pred)
#print(confusion_matrix(val_generator.classes, y_pred))
#print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

## Visualize Model 

In [11]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from tensorflow.keras.utils import to_categorical

In [ ]:
# show the confusion matrix, accuracy, sensitivity, and specificity


# plot the training loss and accuracy
N = EPOCHS 
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

## Save Model 

In [13]:
model.save('/home/gxt/study/CT/models/Inception-V3-50epoch.h5')